# ⊕₄ modes under noise: phase interference and robustness

This notebook compares Level-4 OR operation modes ('quant', 'norm', 'weight', 'opt') across phase differences Δφ and under simple noise channels (phase dephasing and amplitude damping). It plots resulting probabilities F(Δφ) and shows how noise affects interference patterns.

Requirements: numpy, matplotlib, ProbStates >= v0.2 with PhaseRegister extensions.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from probstates import PhaseState, set_phase_or_mode
from probstates.coherence import amp_damp, dephase

np.set_printoptions(precision=4, suppress=True)


In [ ]:
def oplus4_prob(p1, phi1, p2, phi2, mode='quant'):
    set_phase_or_mode(mode)
    a = PhaseState(p1, phi1)
    b = PhaseState(p2, phi2)
    c = a | b
    return c.probability, c.phase

modes = ['quant', 'norm', 'weight', 'opt']

def sweep_delta_phi(p1=0.5, p2=0.5, phi1=0.0, num=361):
    deltas = np.linspace(0.0, 2*np.pi, num)
    results = {mode: [] for mode in modes}
    for d in deltas:
        for mode in modes:
            pr, _ = oplus4_prob(p1, phi1, p2, phi1 + d, mode)
            results[mode].append(pr)
    for mode in modes:
        results[mode] = np.array(results[mode])
    return deltas, results


In [ ]:
# Sweep and plot F(Δφ) for different modes

deltas, results = sweep_delta_phi(p1=0.4, p2=0.6, phi1=0.0, num=361)
plt.figure(figsize=(9,5))
for mode, y in results.items():
    plt.plot(deltas, y, label=mode)
plt.xlabel("Δφ (rad)")
plt.ylabel("F = P( a ⊕₄ b )")
plt.title("⊕₄ modes: probability vs phase difference")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# Noise effect: apply dephasing and amplitude damping to inputs, then combine

def oplus4_with_noise(p1=0.5, p2=0.5, phi1=0.0, phi2=1.0, mode='quant', sigma_phi=0.0, alpha=0.0):
    set_phase_or_mode(mode)
    a = PhaseState(p1, phi1)
    b = PhaseState(p2, phi2)
    if sigma_phi > 0:
        a = dephase(a, sigma_phi=sigma_phi)
        b = dephase(b, sigma_phi=sigma_phi)
    if alpha > 0:
        a = amp_damp(a, alpha)
        b = amp_damp(b, alpha)
    c = a | b
    return c.probability

sigmas = [0.0, 0.2, 0.5]
alpha = 0.3
p1, p2, phi1 = 0.5, 0.5, 0.0
deltas = np.linspace(0.0, 2*np.pi, 181)

plt.figure(figsize=(10,6))
for mode, color in zip(modes, ['C0','C1','C2','C3']):
    for s in sigmas:
        y = [oplus4_with_noise(p1, p2, phi1, phi1 + d, mode=mode, sigma_phi=s, alpha=alpha) for d in deltas]
        plt.plot(deltas, y, color=color, alpha=0.4 + 0.2*(s==0.0), label=f"{mode}, σφ={s}")
plt.xlabel("Δφ (rad)")
plt.ylabel("F with noise")
plt.title("Noise effect on ⊕₄ modes (dephase σφ, amp damp α=0.3)")
plt.legend(ncol=2)
plt.grid(True, alpha=0.3)
plt.show()
